In [2]:
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import math
from random import sample, choice
from sklearn.cluster import AgglomerativeClustering
K=8
N=20
file="parse.res"
# if sys.argc != 1:
#     print("Usage: ./cluster.py result_file")
#     exit()
# file = sys.argv[1]
if not os.path.isabs(file):
    file = os.path.join(os.getcwd(), file)
print(f"Reading {file}...")

content = open(file)
app = []
X = []
for line in content.readlines():
    tmp = line.strip().split(',')
    app.append(tmp[0])
    tmp=[t[:-1] for t in tmp[1:]]
    X.append(tmp)
X=X[:N]
app=app[:N]


def dist(X,Y):
    sum=0
    for i in range(len(X)):
        sum=(float(X[i])-float(Y[i]))*(float(X[i])-float(Y[i]))
    return math.sqrt(sum)

def BuildGraph(limit):
    edges = pd.DataFrame(columns=['sources','targets'])
    N=len(X)
    for i in range(N):
        for j in range(i+1,N):
            if dist(X[i],X[j])<=limit:
                # print(app[i]+","+app[j])
                edges=edges.append({'sources':app[i],'targets':app[j]},ignore_index=True)
                edges=edges.append({'sources':app[j],'targets':app[i]},ignore_index=True)
    # print(edges)
    G=nx.from_pandas_edgelist(edges,source='sources',target='targets')
    for ap in app:
        if ap not in G.nodes:
            G.add_node(ap)
    return G


def Paint(G,name="a"):
    color_map = ['red' if node in ans else 'green' for node in G]
    nx.draw(G,with_labels=True,pos=nx.kamada_kawai_layout(G),node_color=color_map)
    plt.savefig(f'{name}.png')
    plt.clf()

def check(limit):
    G=BuildGraph(limit)
    global ans
    ans=nx.maximal_independent_set(G)
    Paint(G,"paint/"+str(limit))
    return len(ans)>=K

def check_random(limit):
    G=BuildGraph(limit)
    global ans
    ans=sample(app,K)
    Paint(G,"paint/"+str(limit)+"_random")

def check_cluster(limit):
    G=BuildGraph(limit)
    global ans
    ans=[]
    clf = AgglomerativeClustering(n_clusters=K)
    clf.fit(X, app)
    y = clf.fit_predict(X)
    for k in range(K):
        now_app=[]
        for i in range(N):
            if y[i]==k:
                now_app.append(app[i])
        ans.append(choice(now_app))
    Paint(G,"paint/"+str(limit)+"_cluster")

check(1.013)
check_random(1.013)
check_cluster(1.013)
exit()

T=10
Ans=0
Offset=0.1
L=0
R=10000000
while L<=R:
    M=(L+R)/2.0
    if check(M):
        Ans=M
        L=M+1
    else:
        R=M-1

for t in range(T):
    L=0
    R=9
    now=0
    while L<=R:
        M=(L+R)/2.0
        if check(Ans+Offset*M):
            now=M
            L=M+1
        else:
            R=M-1
    Ans=Ans+Offset*now
    Offset=Offset*0.1

print(Ans)
G=BuildGraph(Ans)
Paint(G)


Reading /data/lixiang/mem-pattern-recognition/new_bench/parse.res...


/home/lixiang/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
/home/lixiang/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)


1.1484461921638915


<Figure size 432x288 with 0 Axes>

: 